In [217]:
import pandas as pd
import json
import numpy as np
import os
import glob
import re
import urllib
from bs4 import BeautifulSoup
from pathlib import Path


In [218]:
#def ig_unknown(file, hl_min, hl_max):
#    return "Feature not implemented yet. Write an issue to GitHub."

In [219]:
#def ig_isotope(measured_df, mat_list, hl_min, hl_max, name):
#    precursores = mat_list.split(',')
#    isotopes = []
#    with open("database/irr_products.json") as f:
#        irradiation_products = json.load(f)
#    
#    for precursor in precursores:
#        try:
#            isotopes += irradiation_products[precursor]
#        
#        except:
#            irradiation_products[precursor] = load_irr_prod(precursor)
#            with open("database/irr_products.json", 'w') as json_out:
#                json.dump(irradiation_products, json_out)
#
#    # get unique only
#    isotopes = list(set(isotopes))
#
#    db_parameters = ["half-life", "sigm_half-life", "lambda", "sigm_lambda", "isotope", "Ig", "sigm_Ig", "E_tab", "sigm_E_tab"]#
#
#    measured_df[[db_parameters]] = np.nan*np.ones(shape=(measured_df.shape[0], len(db_parameters)))
#
#    for isotope in isotopes:
#        measured_df = add_db_info(measured_df, isotope)
#    
#    measured_df.to_csv(f"out/{name}wDBinfo.csv")

In [220]:
#def ig_nat_element(file, mat_list, hl_min, hl_max):
#    return "Feature not implemented yet. Write an issue to GitHub."

In [221]:
#def ig_element_spec(file, mat_list, hl_min, hl_max):
#    return "Feature not implemented yet. Write an issue to GitHub."

In [222]:
#def ig_material(file, mat_list, hl_min, hl_max):
#    return "Feature not implemented yet. Write an issue to GitHub."

In [223]:
def append_Igamma_dir(parsed_dir):
    if not os.path.isdir(parsed_dir):
        raise Exception(f"Directory '{parsed_dir}' not found. Consider creating it and moving parsed files there.")

    for parsed_file in glob.iglob(f"{parsed_dir}/*.csv"):
        append_Igamma(parsed_file)

In [ ]:
def permute_columns(df, first_cols):

In [224]:
def append_Igamma(file):
    print(file)
    A, element = re.split(r'(\d+)(\w+)', file)[-3:-1]
    element = element.split("_")[0]
    print(A)
    print(element)

    #Path("./ig_db").mkdir(parents=True, exist_ok=True)
    ig_all_df = pd.read_parquet("ig_db/ig_all.pq")
    print(f"A is {A} and element is {element}")
    ig_df = ig_all_df.loc[f"{A}{element}"]

    #if os.path.isfile(f"ig_db/{A}{element}.csv"):
    #    ig_df = pd.read_csv(f"ig_db/{A}{element}.csv", header=0, index_col=0)
    #    print("Reading the csv with gammas.")
    #else:
    #    Path("./downloads").mkdir(parents=True, exist_ok=True)
    #    if os.path.isfile(f"downloads/{A}{element}.html"):
    #        extract_Igamma(A, element)
    #        ig_df = pd.read_csv(f"ig_db/{A}{element}.csv", header=0, index_col=0)
    #        print("Reading the csv with gammas.")
    #    else:
    #        download(A, element)
    #        extract_Igamma(A, element)
    #        ig_df = pd.read_csv(f"ig_db/{A}{element}.csv", header=0, index_col=0)
    #        print("Reading the csv with gammas.")
    
    parsed_df = pd.read_csv(file, index_col=0)
    joined_df = add_Ig(parsed_df, ig_df)
    
    Path("./with_Ig").mkdir(parents=True, exist_ok=True)
    joined_df = permute_columns(joined_df, ["Energy", "E_tab", "Ig", "Area", "sigm_E", "sigm_Ig", "FWHM", "%err", "Live Time", "Real Time", "Dead Time (rel)"])
    joined_df.to_csv(f"with_Ig/{A}{element}.csv", ignore_index = True)


In [225]:
#df = pd.read_parquet("ig_db/ig_all.pq")

In [226]:
#df.loc["152Eu"]

In [227]:
def add_Ig(df, ig, ig_thr = 1.0):
    #e_col = []
    #ig_col = []

    #a = np.empty((df.shape[0], 2))
    #a[:] = np.nan
    #df[["E_tab", "Ig"]] = a
    # UGLY, never for-loop in pandas
    added_df = df
    #for row_no in range(df.shape[0]):
    for row in df.iterrows():
        # WTF some pandas iterrows bizzare
        row = row[1]
        #suitable_lines = ig[(df["Energy"] - df["FWHM"]).iloc[row_no] > ig["E_tab"] + ig["sigm_E"]]


        #suitable_lines = ig[((df["Energy"] - df["FWHM"]).iloc[row_no] < ig["E_tab"] + ig["sigm_E"]) & ((df["Energy"] + df["FWHM"]).iloc[row_no] > ig["E_tab"] - ig["sigm_E"]) & ig["Ig"] > 0.01*ig_thr]
        #suitable_lines["Energy"] = df["Energy"].iloc[row_no]
        #added_df = added_df.append(suitable_lines)


        suitable_lines = ig[(row["Energy"] - row["FWHM"] < ig["E_tab"] + ig["sigm_E"]) & (row["Energy"] + row["FWHM"] > ig["E_tab"] - ig["sigm_E"]) & ig["Ig"] > 0.01*ig_thr]
        suitable_lines["Energy"] = float(row["Energy"])
        added_df = added_df.append(suitable_lines)

        #for ig_row_no in range(ig.shape[0]):
        #    if (df["Energy"] - df["FWHM"]).iloc[row_no] > ig["E_tab"].iloc[ig_row_no] + ig["sigm_E"].iloc[ig_row_no]:
        #        continue
        #    elif (df["Energy"] + df["FWHM"]).iloc[row_no] < ig["E_tab"].iloc[ig_row_no] - ig["sigm_E"].iloc[ig_row_no]:
        #        break
        #    else:
        #        df.loc[row_no, ["E_tab", "sigm_E_tab", "Ig", "sigm_Ig"]] = ig.loc[ig_row_no]
    print("Ig added")
    return added_df

In [228]:
dd = pd.read_csv("parsed_reports/152Eu_g80.csv", index_col=0)

In [229]:
dd["KKK"] = dd["Energy"][0]

In [230]:
dd["KKK"]

0     25.31
1     25.31
2     25.31
3     25.31
4     25.31
5     25.31
6     25.31
7     25.31
8     25.31
9     25.31
10    25.31
11    25.31
12    25.31
13    25.31
14    25.31
15    25.31
16    25.31
17    25.31
18    25.31
19    25.31
20    25.31
21    25.31
22    25.31
23    25.31
24    25.31
25    25.31
26    25.31
27    25.31
28    25.31
29    25.31
30    25.31
31    25.31
32    25.31
33    25.31
34    25.31
35    25.31
36    25.31
37    25.31
38    25.31
39    25.31
40    25.31
41    25.31
42    25.31
Name: KKK, dtype: float64

In [231]:
for row in df.iterrows():
    print(row[1]["Energy"])

25.31
40.29
75.2
84.81
271.33
275.6
296.09
315.52
324.9
329.56
340.37
357.42
411.14
416.1
443.92
488.62
503.43
533.91
556.48
586.08
656.25
688.37
712.71
778.52
810.03
841.1
866.92
872.56
900.55
918.82
963.56
1004.64
1122.7
1212.56
1233.39
1249.48
1274.19
1292.51
1298.88
1434.25
1457.51
1528.69
1643.53


In [232]:
# ig[((df["Energy"] - df["FWHM"]).iloc[0] < ig["E_tab"] + ig["sigm_E"]) & ((df["Energy"] + df["FWHM"]).iloc[0] > ig["E_tab"] - ig["sigm_E"]) & ig["Ig"] > 0.0001]

In [233]:
# NO need for this now, deprecated

#def getZ(element):
#    element_Z ={
#        "Eu": 63,
#        "U": 92,
#        "Pu": 94
#    }
#    return element_Z[element]

In [234]:
# No need for this now, deprecated

#def download(A, element):
#    url = f"http://nucleardata.nuclear.lu.se/toi/nuclide.asp?iZA={getZ(element)}0{A}"
#    urllib.request.urlretrieve (url, f"downloads/{A}{element}.html")
#    print(f"File {A}{element}.html downloaded.")

In [235]:
#def extract_Igamma(A, element):
#    html_file = open(f"downloads/{A}{element}.html", "r")
#    soup = BeautifulSoup(html_file.read(), 'lxml')
    
#    gammas_table = soup.find_all("table")[4]
#    gammas_rows = gammas_table.find_all('tr')[3:-1]
#    energy = []
#    sigm_energy = []
#    i = []
#    sigm_i = []

#    for row in gammas_rows:
#        cells = row.find_all('td')
        
#        e_val = cells[0].get_text(strip=True)
#        i_val = cells[1].get_text(strip=True)
#        try:
#            ig_val = float(i_val[:-1])
#            sigm_ig_val = float(i_val[-1])
#        except:
#            ig_val = float('NaN')
#            sigm_ig_val = float('NaN')
       
#        energy.append(float(e_val[:-1]))
#        sigm_energy.append(int(e_val[-1]))
#        i.append(ig_val)
        
#        sigm_i.append(sigm_ig_val)


#    df_dict = {
#        "E_tab": energy,
#        "sigm_E_tab": sigm_energy, 
#        "Ig": i,
#        "sigm_Ig": sigm_i
#        }
#    df = pd.DataFrame(df_dict)
#    df_name = f'ig_db/{A}{element}.csv'
#    df.to_csv(df_name)
   
#    print(f"Ig extracted from file 'downloads/{A}{element}.html' into '{df_name}'.")

In [236]:
append_Igamma_dir("parsed_reports")

parsed_reports/152Eu_g80.csv
152
Eu
A is 152 and element is Eu
<ipython-input-227-dd672e0919a4>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  suitable_lines["Energy"] = float(row["Energy"])
<ipython-input-227-dd672e0919a4>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  suitable_lines["Energy"] = float(row["Energy"])
<ipython-input-227-dd672e0919a4>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

S